# Ejercicio 2 - Adrián José Zapater Reig

### Inicialización

In [1]:
! docker run --name mongo_db -it -p 27017:27017 -d mongo:4.0

b536eadf5c0a3f447cfad7ff6a8e3133ace93f4e63d238bb79fe8118150e89bc


In [2]:
! docker cp src/webs.json mongo_db:/tmp/webs.json

In [3]:
! docker exec -it mongo_db mongoimport --db ejercicio2 --collection webs --file /tmp/webs.json

2020-05-01T16:59:06.827+0000	connected to: localhost
2020-05-01T16:59:06.848+0000	imported 51 documents


In [4]:
import pymongo
from pymongo import MongoClient
import pprint
from datetime import datetime

client = MongoClient("localhost", 27017) 
db = client.ejercicio2

In [5]:
# Comprobamos la conexión:
db.list_collection_names()

['webs']

In [6]:
# Mostramos un documento:
db.webs.find_one()

{'_id': ObjectId('5eac555a91f33b532d25b316'),
 'nombre': 'Bertoldo',
 'navegador': 'firefox',
 'fecha': datetime.datetime(2016, 1, 21, 6, 1, 17, 171000),
 'pags': [{'url': 'http://www.vuelosparaAbuelos.es', 'segs': 500},
  {'url': 'https://www.mikokoloko.com', 'segs': 120},
  {'url': 'http://www.comomemongo.com', 'segs': 185},
  {'url': 'https://www.ucm.com', 'segs': 68}]}

In [7]:
# Función auxiliar para imprimir por pantalla los resultados:
def print_query(query_cursor):
    for doc in query_cursor:
        pprint.pprint(doc)

## Consulta 1
Número de sesiones (documentos) en los que se ha visitado la web “https://www.mikokoloko.com”. Nota: si una sesión incluye dos visitas solo se contará una vez.

In [8]:
url = "https://www.mikokoloko.com"
find_json_statement = {"pags.url": url}

In [9]:
# Nomenglatura como en la shell.
# Deprecado.
db.webs.find(find_json_statement).count()

<ipython-input-9-1aef0dccc92f>:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  db.webs.find(find_json_statement).count()


17

In [10]:
db.webs.count_documents(find_json_statement)

17

## Consulta 2
Mostrar los distintos navegadores que se han utilizado en toda la colección.

In [11]:
key = "navegador"
db.webs.find().distinct(key)

['Chrome', 'Explorer', 'firefox']

## Consulta 3
Mostrar únicamente el campo “navegador” de las sesiones realizadas por el usuario “Bertoldo” entre las fechas “20/02/2016” y “25/02/2016” (a las 00:00:00.000 horas en ambos casos).

In [12]:
fields = ["navegador"]
json_find_statement = { 
    "nombre": "Bertoldo", 
    "fecha" : {'$gte': datetime(2016, 2, 20, 6, 0, 0, 0), '$lte': datetime(2016, 2, 25, 0, 0, 0, 0)}
}

query_cursor = db.webs.find(filter=json_find_statement, projection=fields)
print_query(query_cursor)

{'_id': ObjectId('5eac555a91f33b532d25b31b'), 'navegador': 'firefox'}
{'_id': ObjectId('5eac555a91f33b532d25b324'), 'navegador': 'firefox'}
{'_id': ObjectId('5eac555a91f33b532d25b327'), 'navegador': 'firefox'}
{'_id': ObjectId('5eac555a91f33b532d25b32e'), 'navegador': 'firefox'}
{'_id': ObjectId('5eac555a91f33b532d25b332'), 'navegador': 'firefox'}
{'_id': ObjectId('5eac555a91f33b532d25b339'), 'navegador': 'firefox'}
{'_id': ObjectId('5eac555a91f33b532d25b33e'), 'navegador': 'firefox'}


## Agregación 1
Obtener el número de sesiones por cada navegador, ordenadas en orden ascendente. La salida mostrará, para cada navegador, un “_id” que contiene el nombre del navegador, y un campo "total" con el número de sesiones de dicho navegador.

In [13]:
json_aggregate_statement = [
    {
        "$group": {
            "_id" : "$navegador",
            "total": {
                "$sum" : 1
            }
        }
        
    },
    {
        "$sort": {
            "total": 1
        }
    }
]
query_cursor = db.webs.aggregate(json_aggregate_statement)
print_query(query_cursor)

{'_id': 'Chrome', 'total': 14}
{'_id': 'firefox', 'total': 17}
{'_id': 'Explorer', 'total': 20}


## Agregación 2
Obtener la persona con mayor número de sesiones totales en la colección. La salida mostrará para cada persona (sólo 1) un “_id” conteniendo el nombre de la persona y un campo “total” con el número de sesiones totales.

In [14]:
json_aggregate_statement = [
    {
        "$group": {
            "_id" : "$nombre",
            "total": {
                "$sum" : 1
            }
        }
        
    },
    {
        "$sort": {
            "total": -1
        }
    },
    {
        "$limit" : 1
    }
]
query_cursor = db.webs.aggregate(json_aggregate_statement)
print_query(query_cursor)

{'_id': 'Herminia', 'total': 20}


## Agregación 3
Para cada persona indicar el número de URLs visitadas durante más de 100 segundos (se cuentan las repeticiones). La salida mostrará para cada persona un “_id” conteniendo el nombre de la persona y un campo “num” con el número de URLs. Pista: utilizar la opción \\$unwind de aggregate.

In [15]:
json_aggregate_statement = [
    {
        "$unwind": "$pags"
    },
    {
        "$match": { "pags.segs" : { "$gt" : 100}}
    }, 
    {
        "$group": {
            "_id" : "$nombre",
            "num" : { "$sum": 1}
        }
    }
]
query_cursor = db.webs.aggregate(json_aggregate_statement)
print_query(query_cursor)

{'_id': 'Aniceto', 'num': 35}
{'_id': 'Herminia', 'num': 40}
{'_id': 'Bertoldo', 'num': 51}


## Agregación 4
Nombre de la persona que visita más páginas de media por sesión. La salida mostrará un “_id” conteniendo el nombre de la persona y un campo “media” con el valor de la media de páginas visitadas. Pistas: calcular el número de URLs para cada sesión y luego calcular la media de ese dato para cada persona / No hace falta utilizar \\$unwind.

In [16]:
json_aggregate_statement = [
    {
        "$group": {
            "_id" : "$nombre",
            "media" : { "$avg": {"$size": "$pags" }}
        }
    },
    {
        "$sort": { "media" : -1 }
    },
    {
        "$limit": 1
    }
]
query_cursor = db.webs.aggregate(json_aggregate_statement)
print_query(query_cursor)

{'_id': 'Bertoldo', 'media': 4.411764705882353}


## Agregación 5
Calcular el total de sesiones con el navegador “Explorer” en 2016. La salida mostrará el “_id” conteniendo el año (2016) y un campo “total” con el número total de sesiones. Pista: para obtener el año de una fecha se puede utilizar \{\\$year:”\\$fecha”}.

In [17]:
json_aggregate_statement = [
    {
        "$project": {
            "navegador" : 1,
            "year" : {"$year":"$fecha"}
        }
    },
    {
        "$match" : { 
            "navegador" : "Explorer" , 
             "year" : 2016
        }
    },
    {
        "$group" : {
            "_id" : "2016",
            "total" : {"$sum" : 1}
        }
    }
]
query_cursor = db.webs.aggregate(json_aggregate_statement)
print_query(query_cursor)

{'_id': '2016', 'total': 10}


## Agregación 6
Para cada persona calcular el tiempo acumulado de todas las URLs visitadas, en minutos. La salida mostrará un campo “nombre” con el nombre de la persona, y un campo “tiempomin” con el tiempo acumulado. La salida estará ordenada alfabéticamente por los nombres de los usuarios. Pista: utilizar la opción \\$unwind de aggregate.


In [18]:
json_aggregate_statement = [
    {
        "$unwind": "$pags"
    },
    {
        "$group" : {
            "_id" : "$nombre",
            "tiempomin" : { "$sum": {"$multiply" : ["$pags.segs", 60]}}
        }
    },
    {
        "$project": {
            "nombre" : "$_id",
            "_id" : 0,
            "tiempomin" : 1
        }
    },
    {
        "$sort": { "nombre" : 1}
    }
]
query_cursor = db.webs.aggregate(json_aggregate_statement)
print_query(query_cursor)

{'nombre': 'Aniceto', 'tiempomin': 1185660}
{'nombre': 'Bertoldo', 'tiempomin': 948000}
{'nombre': 'Herminia', 'tiempomin': 1986000}


## Agregación 7
Obtener el nombre de las personas que han visitado alguna página que comience por “https”. La salida mostrará para cada persona únicamente el campo “nombre” que contendrá su nombre. Pista: utilizar el operador de proyección \\$substr

In [19]:
json_aggregate_statement = [
    {
        "$unwind": "$pags"
    },
    {
        "$match" : {
            "pags.url" : { "$regex": '^https'}
        }
    },
    {
        "$group" : {
            "_id" : "$nombre"
        }
    },
    {
        "$project": {
            "nombre" : "$_id",
            "_id" : 0
        }
    }
]
query_cursor = db.webs.aggregate(json_aggregate_statement)
print_query(query_cursor)

{'nombre': 'Aniceto'}
{'nombre': 'Bertoldo'}


### Breakdown

In [20]:
! docker stop mongo_db

mongo_db


In [21]:
! docker rm mongo_db

mongo_db
